            BEST FIRST SEARCH

In [13]:
from collections import deque

# Class for the Puzzle node
class PuzzleNode:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action

    def __eq__(self, other):
        return self.state == other.state

    def __hash__(self):
        return hash(str(self.state))

# Function to get possible actions and corresponding new states
def get_possible_actions(state):
    actions = []
    for i in range(len(state)):
        if state[i] == 0:
            row, col = divmod(i, 3)
            if row > 0:
                actions.append(('up', i, i - 3))
            if row < 2:
                actions.append(('down', i, i + 3))
            if col > 0:
                actions.append(('left', i, i - 1))
            if col < 2:
                actions.append(('right', i, i + 1))
            break
    return actions

# Function to perform actions on the state
def apply_action(state, action):
    state = list(state)
    _, i, j = action
    state[i], state[j] = state[j], state[i]
    return tuple(state)

# Function to perform BFS to find the solution
def bfs(start_state, goal_state):
    frontier = deque([PuzzleNode(start_state)])
    explored = set()

    while frontier:
        node = frontier.popleft()
        explored.add(node.state)

        if node.state == goal_state:
            return get_solution_path(node)

        actions = get_possible_actions(node.state)
        for action in actions:
            new_state = apply_action(node.state, action)
            if new_state not in explored:
                new_node = PuzzleNode(new_state, node, action)
                frontier.append(new_node)
                explored.add(new_state)

    return None

# Function to get the solution path
def get_solution_path(node):
    path = []
    while node.parent:
        path.append(node.action)
        node = node.parent
    path.reverse()
    return path

# Function to print the solution path
def print_solution_path(start_state, path):
    current_state = start_state
    print("Start State:")
    print_state(current_state)
    print("Solution Steps:")
    for action in path:
        print(action[0].capitalize() + ":")
        current_state = apply_action(current_state, action)
        print_state(current_state)

# Function to print the state
def print_state(state):
    for i in range(0, 9, 3):
        print(state[i:i+3])

# Example usage
start_state = (1,2,3,8,0,4,7,6,5)  # Initial state of the puzzle
goal_state = (2,8,1,0,4,3,7,6,5)     # Goal state of the puzzle

solution = bfs(start_state, goal_state)

if solution:
    print_solution_path(start_state, solution)
else:
    print("No solution found.")


Start State:
(1, 2, 3)
(8, 0, 4)
(7, 6, 5)
Solution Steps:
Up:
(1, 0, 3)
(8, 2, 4)
(7, 6, 5)
Left:
(0, 1, 3)
(8, 2, 4)
(7, 6, 5)
Down:
(8, 1, 3)
(0, 2, 4)
(7, 6, 5)
Right:
(8, 1, 3)
(2, 0, 4)
(7, 6, 5)
Right:
(8, 1, 3)
(2, 4, 0)
(7, 6, 5)
Up:
(8, 1, 0)
(2, 4, 3)
(7, 6, 5)
Left:
(8, 0, 1)
(2, 4, 3)
(7, 6, 5)
Left:
(0, 8, 1)
(2, 4, 3)
(7, 6, 5)
Down:
(2, 8, 1)
(0, 4, 3)
(7, 6, 5)


         A* algo 

In [15]:

from queue import PriorityQueue

# Calculate the number of correctly placed tiles heuristic
def correctly_placed_tiles_heuristic(current_state, goal_state):
    count = 0
    for i in range(3):
        for j in range(3):
            if current_state[i][j] != goal_state[i][j]:
                count += 1
    return count

# Swap two tiles in the puzzle
def swap_tiles(state, i1, j1, i2, j2):
    new_state = [row[:] for row in state]
    new_state[i1][j1], new_state[i2][j2] = new_state[i2][j2], new_state[i1][j1]
    return new_state

# Get possible moves for the empty tile
def get_possible_moves(state):
    moves = []
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                if i > 0:
                    moves.append(swap_tiles(state, i, j, i - 1, j))
                if i < 2:
                    moves.append(swap_tiles(state, i, j, i + 1, j))
                if j > 0:
                    moves.append(swap_tiles(state, i, j, i, j - 1))
                if j < 2:
                    moves.append(swap_tiles(state, i, j, i, j + 1))
    return moves



def a_star_search(initial_state, goal_state, heuristic):
    priority_queue = PriorityQueue()
    priority_queue.put((0 + heuristic(initial_state, goal_state), 0, initial_state, []))
    visited = set()

    while not priority_queue.empty():
        total_cost, current_cost, current_state, path = priority_queue.get()

        if current_state == goal_state:
            return path

        if tuple(map(tuple, current_state)) in visited:
            continue
        visited.add(tuple(map(tuple, current_state)))

        # Generate all possible moves
        moves = get_possible_moves(current_state)
        for move in moves:
            move_cost = current_cost + 1
            move_heuristic = heuristic(move, goal_state)
            total_cost = move_cost + move_heuristic
            priority_queue.put((total_cost, move_cost, move, path + [move]))

    return None  # Return None if no solution found

# Example usage:
initial_state = [[2, 0, 3], [1, 8, 4], [7, 6, 5]]
goal_state = [[1, 2, 3], [8, 0, 4], [7, 6, 5]]

result = a_star_search(initial_state, goal_state, correctly_placed_tiles_heuristic)
# print("Moves to reach the goal state using A* search:")
# for state in result:
#     print(state)
if result:
    print("Moves to reach the goal state using A* search:")
    for state in result:
        print("State:\n",state)
else:
    print("No solution found.")


Moves to reach the goal state using A* search:
State:
 [[0, 2, 3], [1, 8, 4], [7, 6, 5]]
State:
 [[1, 2, 3], [0, 8, 4], [7, 6, 5]]
State:
 [[1, 2, 3], [8, 0, 4], [7, 6, 5]]
